In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [ ]:
#https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html


In [5]:
test = pd.read_csv('data/ZIP_CBSA_122021.csv', nrows = 100)
test['zip'] = test['zip'].str.slice(start = 1, stop = 6)

test.head()



,zip,cbsa,usps_zip_pref_city,usps_zip_pref_state
0,00683,41900,SAN GERMAN,PR
1,00683,32420,SAN GERMAN,PR
2,00923,41980,SAN JUAN,PR
3,01010,44140,BRIMFIELD,MA
4,01010,49340,BRIMFIELD,MA


In [8]:
hop_proj=sqlite3.connect('data/hop_proj.sqlite')

for chunk in tqdm(pd.read_csv('data/nppes_with_taxonomy.csv', dtype = {'postal_code':str}, chunksize = 1000)):
    chunk['postal_code'] = chunk['postal_code'].str.slice(stop=5)
    chunk.to_sql('nppes', hop_proj, if_exists = 'append', index = False)
    
for chunk in tqdm(pd.read_csv('data/nucc_taxonomy_220.csv', chunksize = 1000)):
    chunk.to_sql('taxonomy', hop_proj, if_exists = 'append', index = False)
    
for chunk in tqdm(pd.read_csv('data/ZIP_CBSA_122021.csv', chunksize = 1000, )):
    chunk['zip'] = chunk['zip'].str.slice(start = 1, stop = 6)
    chunk.to_sql('cbsa', hop_proj, if_exists = 'append', index = False)
    
for chunk in tqdm(pd.read_csv('data/DocGraph_Hop_Teaming_2018_Commercial/DocGraph_Hop_Teaming_2018_Commercial/DocGraph_Hop_Teaming_2018.csv', 
                              dtype = {'zip':str},
                              chunksize = 1000)):
    chunk = chunk[(chunk['transaction_count'] >= 50) & chunk['average_day_wait'] < 50]
    chunk.to_sql('hop', hop_proj, if_exists = 'append', index = False)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [9]:
hop_proj.close()

In [6]:
hop_proj=sqlite3.connect('data/hop_proj.sqlite')

hop_proj.execute('CREATE INDEX npi ON nppes(npi)')
hop_proj.execute('CREATE INDEX postal_code ON nppes(postal_code)')
hop_proj.execute('CREATE INDEX primary_taxonomy ON nppes(primary_taxonomy)')

hop_proj.execute('CREATE INDEX from_npi ON hop(from_npi)')
hop_proj.execute('CREATE INDEX to_npi ON hop(to_npi)')

hop_proj.execute('CREATE INDEX zip ON cbsa(zip)')

hop_proj.execute('CREATE INDEX code ON taxonomy(code)')

hop_proj.close()